# Advent of Code 2022 - Day 11
A* algorithm adapted from: https://medium.com/@nicholas.w.swift/easy-a-star-pathfinding-7e6689c7f7b2

In [ ]:
import string
import numpy as np
import heapq
import math

In [ ]:
from collections import defaultdict

In [ ]:
test_input = """Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi"""

In [ ]:
inp = open("input.txt","r").read()

In [ ]:
height_dict = {x:n for n,x in enumerate(string.ascii_lowercase)}

In [ ]:
d = [[x for x in y] for y in inp.splitlines()]

In [ ]:
S=np.isin(d,'S')
S_loc = (np.nonzero(S)[0][0],np.nonzero(S)[1][0])

In [ ]:
E=np.isin(d,'E')
E_loc = (np.nonzero(E)[0][0],np.nonzero(E)[1][0])

In [ ]:
d[S_loc[0]][S_loc[1]] = 0
d[E_loc[0]][E_loc[1]] = 25

In [ ]:
heights = [[height_dict.get(x,x) for x in a] for a in d]
heights = np.array(heights)

In [ ]:
col = np.array([[100] for x in heights])
heights = np.hstack((col,heights,col))
row = np.array([[100] for x in heights.transpose()] ).transpose()
heights = np.vstack((row,heights,row))

In [ ]:
start = tuple([x + 1 for x in S_loc])
end = tuple([x + 1 for x in E_loc])

In [ ]:
print(start)
print(end)

In [ ]:
maze = heights

In [ ]:
class Node:
    """
    A node class for A* Pathfinding
    """

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position
    
    def __repr__(self):
        return f"{self.position} - g: {self.g} h: {self.h} f: {self.f}"

    # defining less than for purposes of heap queue
    def __lt__(self, other):
          return self.f < other.f
    
    # defining greater than for purposes of heap queue
    def __gt__(self, other):
          return self.f > other.f

In [ ]:
def return_path(current_node):
    path = []
    current = current_node
    while current is not None:
        path.append(current.position)
        current = current.parent
    return path[::-1]  # Return reversed path

In [ ]:
def manhattan_distance(point1, point2):
    distance = 0
    for x1, x2 in zip(point1, point2):
        difference = x2 - x1
        absolute_difference = abs(difference)
        distance += absolute_difference

    return distance

In [ ]:
def part_1(start,end):

    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Heapify the open_list and Add the start node
    heapq.heapify(open_list) 
    heapq.heappush(open_list, start_node)

    # Loop until you find the end
    while len(open_list) > 0:

        # Get the current node
        current_node = heapq.heappop(open_list)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_node:
            #print(return_path(current_node))
            final_path = return_path(current_node)
            return [len(final_path)-1,final_path]

        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]: # Adjacent squares

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])
            #print("node_position is: ",node_position)

            # Make sure climable terrain - if not try next position
            if (maze[node_position[0]][node_position[1]] - maze[current_node.position[0]][current_node.position[1]]) > 1 :
                continue

            # Create new node
            new_node = Node(current_node, node_position)
            #print("new node at: ", new_node.position)

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:

            # Child is on the closed list
            if len([closed_child for closed_child in closed_list if closed_child == child]) > 0:
                continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = math.sqrt(manhattan_distance(child.position,end_node.position))
            child.f = child.g + child.h

            # Child is already in the open list
            if child in open_list: 
                idx = open_list.index(child) 
                if child.g < open_list[idx].g:
                    # update the node in the open list
                    open_list[idx].g = child.g
                    open_list[idx].f = child.f
                open_list[idx].h = child.h
            else:
                # Add the child to the open list
                heapq.heappush(open_list, child)

In [ ]:
p1_ans = part_1(start,end)

### Part 2

need a list of all starting positions with 0 

In [ ]:
starters = []
for i in range(1,heights.shape[0]-1):
    for j in range(1,heights.shape[1]-1):
        if heights[i][j]==0:
            starters.append((i,j))
            

In [ ]:
ans = []
for start in starters:
    path_l = part_1(start,end)
    if path_l != None:
        ans.append(path_l[0])
        print(path_l[0])

This takes an age to run... could defintely optimise by reducing the number of starting positions we check. or just improving part 1.